In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [17]:
import requests
import base64
import json

def get_access_token(client_id, client_secret):
    # Encode as Base64
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode())

    # Make a request for the token
    token_url = "https://accounts.spotify.com/api/token"
    token_data = {
        "grant_type": "client_credentials"
    }
    token_headers = {
        "Authorization": f"Basic {client_creds_b64.decode()}"
    }
    
    r = requests.post(token_url, data=token_data, headers=token_headers)
    if r.status_code not in range(200, 299):
        raise Exception("Could not authenticate client.")
    
    token_response_data = r.json()
    return token_response_data["access_token"]

access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): accounts.spotify.com:443
DEBUG:urllib3.connectionpool:https://accounts.spotify.com:443 "POST /api/token HTTP/1.1" 200 None


In [26]:
def search_for_playlists(access_token, query, total=50):
    playlists = []
    offset = 0
    while len(playlists) < total:
        search_url = f"https://api.spotify.com/v1/search?q={query}&type=playlist&limit=50&offset={offset}"
        search_headers = {
            "Authorization": f"Bearer {access_token}"
        }

        response = requests.get(search_url, headers=search_headers)
        if response.status_code not in range(200, 299):
            break

        search_results = response.json()
        items = search_results['playlists']['items']
        playlists.extend(items)

        if not items:
            break

        offset += 50

    return playlists[:total]

search_terms = ['party', 'classical', 'rock', 'jazz', 'hip hop']
unique_playlists = {}

for term in search_terms:
    playlists = search_for_playlists(access_token, term, 1000 // len(search_terms))
    for playlist in playlists:
        # Use playlist ID as the key to ensure uniqueness
        unique_playlists[playlist['id']] = playlist

# If you need the playlists in list form
all_unique_playlists = list(unique_playlists.values())


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.spotify.com:443
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/search?q=party&type=playlist&limit=50&offset=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.spotify.com:443
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/search?q=party&type=playlist&limit=50&offset=50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.spotify.com:443
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/search?q=party&type=playlist&limit=50&offset=100 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.spotify.com:443
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/search?q=party&type=playlist&limit=50&offset=150 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.spotify.com:443
DEBUG:urllib3.connectionpool:https://api.spotif

In [30]:
len(all_unique_playlists)

982

In [31]:
def get_playlist_tracks(access_token, all_unique_playlists):
    playlist_dictionary = {}

    for playlist in all_unique_playlists:
        playlist_id = playlist['id']  # Assuming each playlist has an 'id' field
        tracks = []

        try:
            # Fetch tracks from the playlist
            playlist_tracks = sp.playlist_tracks(playlist_id)

            # Check if the playlist tracks are successfully retrieved
            if 'items' in playlist_tracks:
                for item in playlist_tracks['items']:
                    track = item.get('track')
                    if track:  # Ensure the track details are available
                        artist_data = [(artist['name'], artist['id']) for artist in track['artists']]
                        track_data = {
                            "artist": artist_data,
                            "song_id": track['id'],
                            "song_name": track['name']
                        }
                        tracks.append(track_data)

        except Exception as e:
            print(f"Error occurred while processing playlist {playlist_id}: {e}")

        playlist_dictionary[playlist_id] = tracks

    return playlist_dictionary

# Example usage
playlist_data = get_playlist_tracks(access_token, all_unique_playlists)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): accounts.spotify.com:443
DEBUG:urllib3.connectionpool:https://accounts.spotify.com:443 "POST /api/token HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.spotify.com
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/67wqv974wApPdKipmNcImY/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/5xS3Gi0fA3Uo6RScucyct6/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/37i9dQZF1DXa2PvUpywmrr/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/37i9dQZF1EIdzRg9sDFEY3/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/play

Error occurred while processing playlist 37i9dQZF1DWTcEjayzrZ4x: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=10)


DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/4T0R5nCnC3m6yZZWQ045pt/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/37i9dQZF1DWXHghfFFOaS6/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/37i9dQZF1DX2WtjAdkiMga/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/4qrHzwnOhHzU0YRtnEMSi0/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/37i9dQZF1DWSSSls9eK29h/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.spotify.com:443 "GET /v1/playlists/37i9dQZF1DWY3X53lmPYk9/tracks?limit=100&offset=0&additional_types=track HTTP/1.1" 200 Non

In [35]:
import pprint
pprint.pprint(playlist_data)

{'00L9Fq1fvpmCk6pjEkh1vZ': [{'artist': [('Lil Mabu', '6FAo7ORAHEzSSf5q10LLfN'),
                                        ('chriseanrock',
                                         '3Idu5nTg2S3wrYwVkPqiwa')],
                             'song_id': '38R5sydMbsBzSPGPMfmtpb',
                             'song_name': 'MR. TAKE YA B*TCH (FEAT. '
                                          'CHRISEANROCK)'},
                            {'artist': [('Drake', '3TVXtAsR1Inumwj472S9r4'),
                                        ('Yeat', '3qiHUAX7zY4Qnjx8TNUzVx')],
                             'song_id': '2YSzYUF3jWqb9YP9VXmpjE',
                             'song_name': 'IDGAF (feat. Yeat)'},
                            {'artist': [('Latto', '3MdXrJWsbVzdn6fe5JYkSQ'),
                                        ('Cardi B', '4kYSro6naA4h99UJvo89HB')],
                             'song_id': '6c6WmIHcHlhccEwSFBhzNa',
                             'song_name': 'Put It On Da Floor Again (feat. '
             

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [43]:
total_length = sum(len(value) for value in playlist_data.values())
    
print(total_length)


73532
